In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
import pyodbc
from sqlalchemy import create_engine
import urllib

In [ ]:
# ar = pd.read_csv('../ar_reports/orig/kiva_ar_2020_02_25.csv')
ar = pd.read_csv('../ar_reports/orig/kiva_ar_2019_12_31.csv')

In [ ]:
len(ar)

In [ ]:
ar.columns

In [ ]:
ar.drop(columns='Unnamed: 16', inplace=True)

In [ ]:
ar.columns = map(str.lower, ar.columns)
ar.columns = ar.columns.str.replace(" ", "_")
ar.rename(columns={'1-30_days': '30_days', 
                   '31-60_days': '60_days', 
                   '61-90_days': '90_days', 
                   '91+_days': 'over_90_days'}, inplace=True)

In [ ]:
ar.replace(" ", 0, inplace=True)

In [ ]:
ar.columns

In [ ]:
ar[['current', '30_days', '60_days', '90_days', 'over_90_days']] = ar[['current', '30_days', '60_days', '90_days', 'over_90_days']].apply(pd.to_numeric)

In [ ]:
# ar[ar['ar_account'] == 'EAST OF EDEN CANNABIS COMPANY'].head(2)

dataframe where state license is null, and where it is not

In [ ]:
ar_null_license = ar[ar['state_license'].isnull()]
ar_full_license = ar[~ar['state_license'].isnull()]

get list of unique account names from df of missing licenses

In [ ]:
ar_missing = ar_null_license['ar_account'].unique()

loop through to create an id for missing ones

In [ ]:
num_vals = len(ar_missing)
array_list = []
max_val = 0
for i in range(num_vals):
    array_list.append("assign_" + str(max_val))
    max_val += 1

put into array the array list that was created above

In [ ]:
id_array = np.asarray(array_list)

match the unique missing accounts with the unqiue id

In [ ]:
pd_miss_fill = pd.DataFrame({'ar_account': ar_missing, 'state_license': id_array})

drop the state license from the main df (that has all missing rows, not just unique) to then be merged with the above

In [ ]:
ar_null_pd = ar_null_license.drop(columns='state_license')
ar_fill = ar_null_pd.merge(pd_miss_fill, on='ar_account', how='left')

combine the missing ones to the non-missing ones

In [ ]:
ar_final = ar_full_license.append(ar_fill, sort=False, ignore_index=True)

In [ ]:
len(ar_final)

number of iterations of each payor. should this be done at the end?

In [ ]:
# ar_final.groupby('ar_account', as_index=False)['state_license'].count()

In [ ]:
count_payor = ar_final.groupby('ar_account', as_index=False)['state_license'].count()
count_payor.rename(columns={'state_license': 'count_payor'}, inplace=True)

In [ ]:
ar_final.rename(columns={'_original_amount_': 'orig_amt', '_balance_outstanding_': 'bal_out'}, inplace=True)

In [ ]:
ar_final[ar_final['ar_account'] == 'EAST OF EDEN CANNABIS COMPANY'].head(2)

In [ ]:
sum_days_over = ar_final.groupby('ar_account', as_index=False)['days_over_due'].sum()
sum_orig_amt_clean = ar_final.groupby('ar_account', as_index=False)['orig_amt'].sum()
sum_bal_out_clean = ar_final.groupby('ar_account', as_index=False)['bal_out'].sum()
sum_current_clean = ar_final.groupby('ar_account', as_index=False)['current'].sum()
sum_30_day_clean = ar_final.groupby('ar_account', as_index=False)['30_days'].sum()
sum_60_day_clean = ar_final.groupby('ar_account', as_index=False)['60_days'].sum()
sum_90_day_clean = ar_final.groupby('ar_account', as_index=False)['90_days'].sum()
sum_over_90_day_clean = ar_final.groupby('ar_account', as_index=False)['over_90_days'].sum()

In [ ]:
count_payor.head(1)

In [ ]:
data_frames = [count_payor, sum_days_over, sum_orig_amt_clean, sum_bal_out_clean, sum_current_clean, sum_30_day_clean,
               sum_60_day_clean, sum_90_day_clean, sum_over_90_day_clean]

In [ ]:
df_sums = reduce(lambda left,right: pd.merge(left,right,on=['ar_account'], how='outer'), data_frames)

In [ ]:
df_sums.rename(columns={'days_over_due': 'sum_days_over', 
                        'orig_amt': 'sum_orig_amt',
                        'bal_out': 'sum_bal_out', 
                        'current': 'sum_current',
                        '30_days': 'sum_30_day', 
                        '60_days': 'sum_60_day',
                        '90_days': 'sum_90_day', 
                        'over_90_days': 'sum_over_90'
}, inplace=True)

In [ ]:
# ar_final[['state_license', 'ar_account', 'company']].iloc[1448]

In [ ]:
ar_to_merge = ar_final[['state_license', 'ar_account', 'company']]

In [ ]:
# ar_to_merge[ar_to_merge['state_license'] == 'C10-0000224-LIC']

In [ ]:
# ar_to_merge[['state_license', 'ar_account', 'company']].iloc[1448]

In [ ]:
# ar_to_merge[ar_to_merge.duplicated()]
pd.set_option('mode.chained_assignment', None)

ar_to_merge.drop_duplicates(subset='ar_account', keep='first', inplace=True)

In [ ]:
# ar_to_merge[ar_to_merge['ar_account'] == 'EAST OF EDEN CANNABIS COMPANY'].head(2)

In [ ]:
df_sums = pd.merge(df_sums, ar_to_merge, on='ar_account', how='outer')

In [ ]:
# df_sums[df_sums['ar_account'] == 'EAST OF EDEN CANNABIS COMPANY']

### if the balance outstanding is less than 500, then it is 0, then the whole row is 0

making a copy to compare original vs edited at the end

In [ ]:
df_sums['sum_orig_amt_v2'] = df_sums['sum_orig_amt']
df_sums['sum_bal_out_v2'] = df_sums['sum_bal_out']
df_sums['sum_current_v2'] = df_sums['sum_current']
df_sums['sum_30_day_v2'] = df_sums['sum_30_day']
df_sums['sum_60_day_v2'] = df_sums['sum_60_day']
df_sums['sum_90_day_v2'] = df_sums['sum_90_day']
df_sums['sum_over_90_v2'] = df_sums['sum_over_90']

find index of all the instances where the balance outstanding is less than 0 (includes negatives)

In [ ]:
list2 = df_sums[df_sums['sum_bal_out_v2'] < 0].index

In [ ]:
pd.set_option('mode.chained_assignment', None)

df_sums['sum_orig_amt_v2'].loc[list2] = 0
df_sums['sum_bal_out_v2'].loc[list2] = 0
df_sums['sum_current_v2'].loc[list2] = 0
df_sums['sum_30_day_v2'].loc[list2] = 0
df_sums['sum_60_day_v2'].loc[list2] = 0
df_sums['sum_90_day_v2'].loc[list2] = 0
df_sums['sum_over_90_v2'].loc[list2] = 0

### if the original amount is 0 and the balance outstanding is over 500, then mark as "Delinquent"

change to write off or credit loss

In [ ]:
# list1 = df_sums[(df_sums['sum_orig_amt'] == 0) & (df_sums['sum_bal_out'] > 0)].index

In [ ]:
# df_sums['delinquent'] = 0

In [ ]:
# pd.set_option('mode.chained_assignment', None)

# df_sums['delinquent'].loc[list1] = 1

In [ ]:
# df_sums[df_sums['delinquent'] == 1]

The two steps above have removed all negatives from original / outstanding and if one is 0 then it's either delinquent or they are both 0

### apply negatives to the positive in the days past due

looping through columns going right to left to remove negatives

In [ ]:
for x in range(len(df_sums)):
    if df_sums['sum_over_90_v2'].iloc[x] < 0:
        val1 = df_sums['sum_over_90_v2'].iloc[x] + df_sums['sum_90_day_v2'].iloc[x]
        if val1 < 0:
            val2 = df_sums['sum_60_day_v2'].iloc[x] + val1
            if val2 < 0:
                val3 = df_sums['sum_30_day_v2'].iloc[x] + val2
                if val3 < 0:
                    val4 = df_sums['sum_current_v2'].iloc[x] + val3
                    df_sums['sum_over_90_v2'].iloc[x] = 0
                    df_sums['sum_90_day_v2'].iloc[x] = 0
                    df_sums['sum_60_day_v2'].iloc[x] = 0
                    df_sums['sum_30_day_v2'].iloc[x] = 0
                    df_sums['sum_current_v2'].iloc[x] = val4
                else:
                    df_sums['sum_over_90_v2'].iloc[x] = 0
                    df_sums['sum_90_day_v2'].iloc[x] = 0
                    df_sums['sum_60_day_v2'].iloc[x] = 0
                    df_sums['sum_30_day_v2'].iloc[x] = val3
            else:
                df_sums['sum_over_90_v2'].iloc[x] = 0
                df_sums['sum_90_day_v2'].iloc[x] = 0
                df_sums['sum_60_day_v2'].iloc[x] = val2
        else:
            df_sums['sum_over_90_v2'].iloc[x] = 0
            df_sums['sum_90_day_v2'].iloc[x] = val1
                    

In [ ]:
for x in range(len(df_sums)):
    if df_sums['sum_90_day_v2'].iloc[x] < 0:
        val1 = df_sums['sum_90_day_v2'].iloc[x] + df_sums['sum_60_day_v2'].iloc[x]
        if val1 < 0:
            val2 = df_sums['sum_30_day_v2'].iloc[x] + val1
            if val2 < 0:
                val3 = df_sums['sum_current_v2'].iloc[x] + val2
                df_sums['sum_90_day_v2'].iloc[x] = 0
                df_sums['sum_60_day_v2'].iloc[x] = 0
                df_sums['sum_30_day_v2'].iloc[x] = 0
                df_sums['sum_current_v2'].iloc[x] = val3
            else:
                df_sums['sum_90_day_v2'].iloc[x] = 0
                df_sums['sum_60_day_v2'].iloc[x] = 0
                df_sums['sum_30_day_v2'].iloc[x] = val2
        else:
            df_sums['sum_90_day_v2'].iloc[x] = 0
            df_sums['sum_60_day_v2'].iloc[x] = val1
          

In [ ]:
for x in range(len(df_sums)):
    if df_sums['sum_60_day_v2'].iloc[x] < 0:
        val1 = df_sums['sum_60_day_v2'].iloc[x] + df_sums['sum_30_day_v2'].iloc[x]
        if val1 < 0:
            val2 = df_sums['sum_current_v2'].iloc[x] + val1
            df_sums['sum_60_day_v2'].iloc[x] = 0
            df_sums['sum_30_day_v2'].iloc[x] = 0
            df_sums['sum_current_v2'].iloc[x] = val2
        else:
            df_sums['sum_60_day_v2'].iloc[x] = 0
            df_sums['sum_30_day_v2'].iloc[x] = val1

In [ ]:
for x in range(len(df_sums)):
    if df_sums['sum_30_day_v2'].iloc[x] < 0:
        val1 = df_sums['sum_30_day_v2'].iloc[x] + df_sums['sum_current_v2'].iloc[x]
        df_sums['sum_30_day_v2'].iloc[x] = 0
        df_sums['sum_current_v2'].iloc[x] = val1

looping through left to right. all negatives will have been moved to current from above steps

In [ ]:
for x in range(len(df_sums)):
    if df_sums['sum_current_v2'].iloc[x] < 0:
        val1 = df_sums['sum_current_v2'].iloc[x] + df_sums['sum_30_day_v2'].iloc[x]
        if val1 < 0:
            val2 = df_sums['sum_60_day_v2'].iloc[x] + val1
            if val2 < 0:
                val3 = df_sums['sum_90_day_v2'].iloc[x] + val2
                if val3 < 0:
                    val4 = df_sums['sum_over_90_v2'].iloc[x] + val3
                    df_sums['sum_current_v2'].iloc[x] = 0
                    df_sums['sum_30_day_v2'].iloc[x] = 0
                    df_sums['sum_60_day_v2'].iloc[x] = 0
                    df_sums['sum_90_day_v2'].iloc[x] = 0
                    df_sums['sum_over_90_v2'].iloc[x] = val4
                else:
                    df_sums['sum_current_v2'].iloc[x] = 0
                    df_sums['sum_30_day_v2'].iloc[x] = 0
                    df_sums['sum_60_day_v2'].iloc[x] = 0
                    df_sums['sum_90_day_v2'].iloc[x] = val3
            else:
                df_sums['sum_current_v2'].iloc[x] = 0
                df_sums['sum_30_day_v2'].iloc[x] = 0
                df_sums['sum_60_day_v2'].iloc[x] = val2
        else:
            df_sums['sum_current_v2'].iloc[x] = 0
            df_sums['sum_30_day_v2'].iloc[x] = val1
                    

test to see if there are still any negatives in the DPD columns

In [ ]:
# df_sums.iloc[:,14:19]

In [ ]:
list1 = df_sums.iloc[:,14:19].apply(lambda x: pd.Series([(x < 0).sum()]), axis=1)
max(list1)

In [ ]:
df_sums.columns

In [ ]:
df_final = df_sums[[
    'state_license', 
    'ar_account', 
    'company',     
    'count_payor', 
    'sum_days_over',
    'sum_orig_amt_v2',
    'sum_bal_out_v2', 
    'sum_current_v2', 
    'sum_30_day_v2', 
    'sum_60_day_v2',
    'sum_90_day_v2', 
    'sum_over_90_v2'
#     'delinquent'
]]

the goal is to match to a company id, and if there isn't one to create one or find it?

In [ ]:
df_final['date_file'] = pd.datetime(2019, 12, 31)
df_final['client'] = 'Kiva'

In [ ]:
len(df_final)

In [ ]:
df_final.head(1)

In [ ]:
df_final.drop(columns='company', inplace=True)

In [ ]:
df_final.rename(columns={'state_license': 'license_number', 
           'sum_orig_amt_v2': 'orig_amt', 
          'sum_bal_out_v2': 'bal_out', 
          'sum_current_v2': 'current', 
          'sum_30_day_v2': '30_days',
           'sum_60_day_v2': '60_days', 
          'sum_90_day_v2': '90_days', 
          'sum_over_90_v2': 'over_90_days', 
           'date_file': 'date'}, inplace=True)

In [ ]:
df_final.head(1)

In [ ]:
df_final['summary'] = 0

In [ ]:
df_final['license_provided'] = 0

In [ ]:
len(df_final.columns)

In [ ]:
df_final = df_final[['license_number', 
          'license_provided',
          'ar_account', 
          'count_payor', 
          'sum_days_over',
           'orig_amt', 
          'bal_out', 
          'current', 
          '30_days', 
          '60_days', 
          '90_days',
           'over_90_days', 
#           'delinquent', 
          'date', 
          'client', 
          'summary'
       ]]

In [ ]:
df_delinq = pd.read_csv('../ar_reports/orig/no_neg/old/kiva_delinq_12_31_19.csv')

In [ ]:
len(df_delinq)

In [ ]:
len(df_final)

In [ ]:
df_final['delinquent'] = df_final['ar_account'].isin(df_delinq['ar_account']).astype(int)

In [ ]:
len(df_final[df_final['delinquent'] == 1])

In [ ]:
df_final.to_csv('../ar_reports/orig/no_neg/kiva_ar_no_neg_12_31_v2.csv', index=False)

In [ ]:
# df_final.to_excel('../ar_reports/kiva_ar_edited_2020_03_25.xlsx', index=False)

In [ ]:
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=ca_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")

In [ ]:
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))

In [ ]:
df_final.to_sql('ar_report', con=engine, if_exists='append', index=False)

# Notes

In [ ]:
# cnxn = pyodbc.connect('Trusted_Connection=yes',
#                       server = 'DESKTOP-KA6KCMH\SQLEXPRESS', 
#                       driver = '{ODBC Driver 17 for SQL Server}',
#                       database = 'ca_cannabis_v3'
#                       )

In [ ]:
# engine = create_engine('mssql+pyodbc://DESKTOP-KA6KCMH\SQLEXPRESS/ca_cannabis_v3?driver=ODBC Driver 17 for SQL Server')

In [ ]:
# ar['30_day_clean'] = ar['30_days'] 

# ar['30_day_clean'] = ar['30_day_clean'].str.replace("(", "-")
# ar['30_day_clean'] = ar['30_day_clean'].str.replace(")", "")
# ar['30_day_clean'] = ar['30_day_clean'].str.replace(",", "")
# ar['30_day_clean'] = ar['30_day_clean'].str.replace("$", "")

# ar['30_day_clean'] = ar['30_day_clean'].astype(float)

# v1 = ar.columns.get_loc("30_day_clean")
# ar['30_day_binary'] = ar.iloc[:,v1].apply(lambda x: 1 if x > 0 else 0)

# ar['60_day_clean'] = ar['60_days'] 

# ar['60_day_clean'] = ar['60_day_clean'].str.replace("(", "-")
# ar['60_day_clean'] = ar['60_day_clean'].str.replace(")", "")
# ar['60_day_clean'] = ar['60_day_clean'].str.replace(",", "")
# ar['60_day_clean'] = ar['60_day_clean'].str.replace("$", "")

# ar['60_day_clean'] = ar['60_day_clean'].astype(float)

# v1 = ar.columns.get_loc("60_day_clean")
# ar['60_day_binary'] = ar.iloc[:,v1].apply(lambda x: 1 if x > 0 else 0)

# ar['90_day_clean'] = ar['90_days'] 

# ar['90_day_clean'] = ar['90_day_clean'].str.replace("(", "-")
# ar['90_day_clean'] = ar['90_day_clean'].str.replace(")", "")
# ar['90_day_clean'] = ar['90_day_clean'].str.replace(",", "")
# ar['90_day_clean'] = ar['90_day_clean'].str.replace("$", "")

# ar['90_day_clean'] = ar['90_day_clean'].astype(float)

# v1 = ar.columns.get_loc("90_day_clean")
# ar['90_day_binary'] = ar.iloc[:,v1].apply(lambda x: 1 if x > 0 else 0)

# ar['over_90_day_clean'] = ar['over_90_days'] 

# ar['over_90_day_clean'] = ar['over_90_day_clean'].str.replace("(", "-")
# ar['over_90_day_clean'] = ar['over_90_day_clean'].str.replace(")", "")
# ar['over_90_day_clean'] = ar['over_90_day_clean'].str.replace(",", "")
# ar['over_90_day_clean'] = ar['over_90_day_clean'].str.replace("$", "")

# ar['over_90_day_clean'] = ar['over_90_day_clean'].astype(float)

# v1 = ar.columns.get_loc("over_90_day_clean")
# ar['over_90_day_binary'] = ar.iloc[:,v1].apply(lambda x: 1 if x > 0 else 0)

# ar['orig_amt_clean'] = ar['original_amount'] 

# ar['orig_amt_clean'] = ar['orig_amt_clean'].str.replace("(", "-")
# ar['orig_amt_clean'] = ar['orig_amt_clean'].str.replace(")", "")
# ar['orig_amt_clean'] = ar['orig_amt_clean'].str.replace(",", "")
# ar['orig_amt_clean'] = ar['orig_amt_clean'].str.replace("$", "")

# ar['orig_amt_clean'] = ar['orig_amt_clean'].astype(float)

# v1 = ar.columns.get_loc("orig_amt_clean")
# ar['orig_amt_clean_binary'] = ar.iloc[:,v1].apply(lambda x: 1 if x > 0 else 0)

# ar['bal_out_clean'] = ar['balance_outstanding'] 

# ar['bal_out_clean'] = ar['bal_out_clean'].str.replace("(", "-")
# ar['bal_out_clean'] = ar['bal_out_clean'].str.replace(")", "")
# ar['bal_out_clean'] = ar['bal_out_clean'].str.replace(",", "")
# ar['bal_out_clean'] = ar['bal_out_clean'].str.replace("$", "")

# ar['bal_out_clean'] = ar['bal_out_clean'].astype(float)

# v1 = ar.columns.get_loc("bal_out_clean")
# ar['bal_out_clean_binary'] = ar.iloc[:,v1].apply(lambda x: 1 if x > 0 else 0)


# ar['current_clean'] = ar['current'] 

# ar['current_clean'] = ar['current_clean'].str.replace("(", "-")
# ar['current_clean'] = ar['current_clean'].str.replace(")", "")
# ar['current_clean'] = ar['current_clean'].str.replace(",", "")
# ar['current_clean'] = ar['current_clean'].str.replace("$", "")

# ar['current_clean'] = ar['current_clean'].astype(float)

# v1 = ar.columns.get_loc("current_clean")
# ar['current_clean_binary'] = ar.iloc[:,v1].apply(lambda x: 1 if x > 0 else 0)